# Lecture 11 - Number Representation and Precision + HW 5

Real numbers are stored with a decimal precision (or mantissa) and the decimal exponent range. The mantissa contains the significant figures of the number (and thereby the precision of the number). A number like (9.90625)10 in the decimal representation is given in a binary representation by

(1001.11101)$_2$ = $1\times2^3 +0\times2^2 +0\times2^1 +1\times2^0 +1\times2^{−1} +1\times2^{−2} +1\times2^{−3} +0\times2^{−4} +1 \times 2^{−5}$

and it has an exact machine number representation since we need a finite number of bits to represent this number. This representation is however not very practical. Rather, we prefer to use a scientific notation. In the decimal system we would write a number like 9.90625 in what is called the normalized scientific notation. This means simply that the decimal point is shifted and appropriate powers of 10 are supplied. Our number could then be written as
$9.90625 = 0.990625 \times 10^1$,
and a real non-zero number could be generalized as
$x = \pm r \times 10^n$,
with a $r$ a number in the range $1/10 \le r < 1$. In a similar way we can represent a binary number in
scientific notation as
$x = \pm q \times 2^m$,
with a $q$ a number in the range $1/2 \le q < 1$.

In a typical computer, floating-point numbers are represented in the way described above, but with certain restrictions on q and m imposed by the available word length. In the machine, our number x is represented as

$x = (−1)^s \times mantissa \times 2^{exponent}$

where $s$ is the sign bit, and the exponent gives the available range. With a single-precision word, 32 bits, 8 bits would typically be reserved for the exponent, 1 bit for the sign and 23 for the mantissa. 

## 32-bit – single precision (old computers):

Sign bit: 1 bit

Exponent: 8 bits

Significand precision: 24 bits (23 explicitly stored)

This gives 6–9 significant decimal digits precision!

## 64-bit = double precision (normal modern computers):

Sign bit: 1 bit

Exponent: 11 bits

Significand precision: 53 bits (52 explicitly stored)

This gives 15–17 significant decimal digits precision.
This the the Python default standard


## 128-bit = quadruple precision:

Sign bit: 1 bit

Exponent: 15 bits

Significand precision: 113 bits (112 explicitly stored)

This gives 33–36 significant decimal digits precision.


## 256-bit – Octuple precision:

Sign bit: 1 bit
    
Exponent: 19 bits
    
Significand precision: 237 bits (236 explicitly stored)

THIS IS RARELY IMPLEMENTED

Max unsigned integer is: 115792089237316195423570985008687907853269984665640564039457584007913129639935



In [1]:
a = 115792089237316195423570985008687907853269984665640564039457584007913129639935
print(a+1)

115792089237316195423570985008687907853269984665640564039457584007913129639936


In [4]:
print(a**20)

1877490722242957624872829180435341496724700098100283274460625329492637173680945959234650087277370147691301609484110510594855641641821538741969504355528797186243654233136942273904508513303437116899358481348014544794522363835441750692307861565626734781272878346295611364152823220618849708635609579439804734971617950670948393727131952778781266075191018642163576820272942470884887099137593064465197866403146844940375791754227590550801195843174756792737078420047699379647736516892037616624970069061927811947126651174502341975015811597004574357763055321007279539991827437373211748969574024034812257751086229003662477681956135836330777334986896577187190738942426408936223589099193913104193637443028886131064124945856631241588733221587018658338148012000785661692542650580584015826087694558993241441546701684739602304039167216806209240321407026854635073748059431163666439930583905809732185343232089933108898599823099623124105764062825056087838011386118515715541140641109551069442563853590409177801342683634441

Python 3 has NO real interger limit length!! 

# Precision effects

One important consequence of rounding error is that you should **NEVER Use an if statment to test equality of two floats.**  For instance, you should nerev, in any program, have a statment like:

In [ ]:
x = 3 * 1.1
if x == 3.3:
    print("x = ",x,"and we have trigged the proper logic")
else:
    print("What is x really :", x)

If you need to do a logic trigger based on a float:

In [ ]:
epsilon = 1e-12
if abs(x-3.3) < epsilon:
    print("x = ",x,"and we have trigged the proper logic")
else:
    print("what is x really :", x)

## Which operations are most important in dealing with precision?

__Subtraction__ and __Derivatives__

## Subtraction

a = b - c

We have:   $fl(a) = fl(b) - fl(c) = a(1+\epsilon_a)$  or
            $fl(a) = b(1+\epsilon_b) - c(1+\epsilon_c)$
            
So, $fl(a)/a = 1 + \epsilon_b (b/a) - \epsilon_c (c/a)$

IF $b \sim c$, we have the potential of increased error on $fl(a)$


If we have:

$x = 1000000000000000$

$y = 1000000000000001.2345678901234$

as far the computer is concerned:
    

In [5]:
x = 1000000000000000.0000000000000
y = 1000000000000001.2345678901234
 
print(y-x) 


1.25


**The true result should be 1.2345678901234!**

In other words, instead of 16-figure accuracy we now only have three figures and the fractional error is a few percent of the true value.  This is much worse than before!


To see another exanple of this in practice, consider two numbers:

$x = 1$, and $ y = 1+10^{-14}\sqrt 2$ 

Simply we can see that:

$ 10^{14} (y - x) = \sqrt 2$

Let us try the same calculation in python:
 

In [6]:
from math import sqrt
x = 1.0
y = 1.0 + (1e-14)*sqrt(2)

print((1e14)*(y-x))
print(sqrt(2))
print("Difference is:",(1e14)*(y-x)-sqrt(2))


1.4210854715202004
1.4142135623730951
Difference is: 0.006871909147105226


Again error off by a percent.  We need to be careful in how we code math!

## Example 1:  Summing $1/n$ 

Consider the series:

$$s_1 = \sum_{n=1}^N \frac{1}{n}$$ which is finite when N is finite, then consider

$$s_2 = \sum_{n=N}^1 \frac{1}{n}$$ which when summed analyitically should give $s_2 = s_1$

Write a code to perform both of these to sums for N = 1e8 and compare

In [10]:
s1, s2 = 0, 0

N = 1e8

# Write a code to perform both of these to sums for N = 1e8 and compare
for x in range(1, int(N+1)):
    s1 += 1/x
    
for x in range(1, int(N+1)):
    s2 += 1/(N-(x-1))

print(s1, s2, "Diff:", s1-s2)


18.997896413852555 18.997896413853447 Diff: -8.917311333789257e-13


## Example 2: $e^{-x}$

There are three possible algorithms for $e^{-x}$

1) **Simple:** $$e^{-x} = \sum_{n=0}^{\infty} (-1)^n \; \frac{x^n}{n!}$$  


2) **Inverse:**  $$e^{x} = {\sum_{n=0}^{\infty} \frac{x^n}{n!}}$$  Then take the inverse:   $$e^{-x} = \frac{1}{e^{x}}$$


3) **Recursion: (see example below)** $$e^{-x} = \sum_{n=0}^{\infty} s_n = \sum_{n=0}^{\infty} (-1)^n \; \frac{x^n}{n!}$$  where  $$ s_n = -s_{n-1} \frac{x}{n}$$ and $$s_0 = 1$$




In [33]:
import numpy as np
np.exp(-1)

# write a function to compute e^-X for all three methods 
# Then chack their output for x = 0 - 100, in steps of 10 and 
# Compare to the numpy version of exp(-x) which is imported above. 
def e_minusx_simple(x):
    # code here
    emxsmp = 1
    ifact = 1
    for i in range(1, 5000):
        ifact *= i
        emxsmp += ((-1)**i) * (x**i)/(ifact)
    
    return emxsmp

def e_minusx_inverse(x):
    # code here
    emxinvA = 1
    ifact = 1
    for i in range(1, 5000):
        ifact *= i
        emxinvA += (x**i)/(ifact)
    emxinv = 1 / emxinvA
    
    return emxinv

# note use a function for s_n (See example below)
def s_n(x, n):
    output = 0
    if n > 0:
        output = (-1) * s_n(x, n-1) * x / n
    else:
        output = 1
    return output
    

def e_minusx_recurse(x):
    # code here
    emxrec = 1
    for i in range(1, 1000):
        emxrec += s_n(x, i)
    
    return emxrec


# main code here
for x in range(1, 100, 10):
    print("'x' simple inverse recurse numpy")
    print("--- ------ ------- ------- -----")
    print(x,",", e_minusx_simple(x),",", e_minusx_inverse(x),",", e_minusx_recurse(x),",", np.exp(-1*x))
    print("\n")

'x' simple inverse recurse numpy
--- ------ ------- ------- -----
1 , 0.36787944117144245 , 0.3678794411714423 , 0.36787944117144245 , 0.36787944117144233


'x' simple inverse recurse numpy
--- ------ ------- ------- -----
11 , 1.670170206765736e-05 , 1.670170079024566e-05 , 1.6701701266058984e-05 , 1.670170079024566e-05


'x' simple inverse recurse numpy
--- ------ ------- ------- -----
21 , 4.67354049567923e-09 , 7.58256042791191e-10 , -3.1648595607706816e-09 , 7.582560427911907e-10


'x' simple inverse recurse numpy
--- ------ ------- ------- -----
31 , -0.00015587898712317639 , 3.442477108469977e-14 , 0.0003068088967300021 , 3.442477108469977e-14


'x' simple inverse recurse numpy
--- ------ ------- ------- -----
41 , -5.814920105652159 , 1.5628821893349876e-18 , -1.3889553949418647 , 1.5628821893349888e-18


'x' simple inverse recurse numpy
--- ------ ------- ------- -----
51 , 76244.20932341686 , 7.095474162284701e-23 , -94010.44750307851 , 7.095474162284704e-23


'x' simple inve

### Recursion Example

In [ ]:
def factorial(x):
    """This is a recursive function to find the factorial of an integer"""
    if x == 1:
        return 1
    else:
        return (x * factorial(x-1))
# Testing it below.
num = 5
print("The factorial of", num, "is", factorial(num))

## Homework #5 

**REMINDER:** *All coding assignment will be turned in as .ipynb files, to the same PHYS_X0223 repository on GitHub.*   
*They should be turned in with the following naming:*
    
    Lastname_Firstinitial_23_HW#.ipynb
    

**The semi-empirical mass formula**

In nuclear physics, the semi-empirical mass formula is a formula for calculating the
approximate nuclear binding energy $B$ of an atomic nucleus with atomic number $Z$
and mass number $A$. The formula looks like this:
    
$$ B = a_1 A - a_2 A^{2/3} - a_3 \frac{Z^2}{A^{1/3}} - a_4 \frac{(A - 2Z)^2}{A} - \frac{a_5}{A^{1/2}} $$

where, in units of millions of electron volts (MeV), the constants are $a_1 =
15.67$, $a_2 = 17.23$, $a_3 = 0.75$, $a_4 = 93.2$, and

$$ a_5  \; =  \;\; \left\{ \begin{array} {r@{\quad\tt if \quad}l} 0 & A \;{\tt is
      \; odd}, \\
    12.0 & A \;{\tt and}\; Z \;{\tt are \;both \;even}, \\ -12.0 & A \;{\tt is
     \;  even \; and}\;  Z \;{\tt is
  \;  odd.} \end{array} \right. $$

Write a **function** that takes as its input the values of $A$ and $Z$, and
prints out: 
* (a) the binding energy $B$ for the corresponding atom and 
* (b) the binding energy per nucleon, which is $B/A$. 

Use your program to find
the binding energy of an atom with $A = 58$ and $Z = 28$. (Hint: The
correct answer is around 490 MeV.) 





In [17]:
def binding_energy(A, Z):
    B = (15.67*A) - (17.23*(A**(2/3))) - (0.75*Z*Z/(A**(1/3))) - (93.2*((A-(2*Z))**2)/A)
    
    if A % 2 == 1:
        return B
        
    elif Z % 2 == 0:
        B -= 12/(A**0.5)
        return B
        
    else:
        B += 12/(A**0.5)
        return B
        
print(binding_energy(58, 28))

490.78425241273493


Also run for,  $A = 59$ and $Z = 28$ **AND** $A = 58$ and $Z = 27$.

In [18]:
print(binding_energy(59, 28))
print(binding_energy(58, 27))

498.144677545714
485.30934897614435
